## 导入必要的库

In [1]:
%matplotlib inline

import numpy as np
from math import *
from matplotlib import pyplot as plt

In [2]:
def loadDataSet():
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1]    #1 is abusive, 0 not
    return postingList,classVec

## 根据样本生成词向量
这里使用map也可以，使用set也可以

In [3]:
def buildWordVec(wordLists):
    wordVec = set([])
    for words in wordLists:
        wordVec = wordVec | set(words)
    return list(wordVec)

In [4]:
dataset, datalabel = loadDataSet()
wordVec = buildWordVec(dataset)
print(wordVec)

['park', 'posting', 'problems', 'dog', 'food', 'stupid', 'has', 'is', 'mr', 'how', 'help', 'steak', 'not', 'so', 'dalmation', 'quit', 'garbage', 'ate', 'to', 'maybe', 'him', 'flea', 'buying', 'love', 'stop', 'I', 'please', 'licks', 'take', 'worthless', 'cute', 'my']


## 把句子转换为词向量
使用list

In [5]:
def setOfWordVec(wordVec, words):
    N = len(wordVec)
    res = [0]*N
    for word in words:
        if word in wordVec:
            res[wordVec.index(word)] = 1
    return res

In [6]:
test_words = ['my', 'dog', 'is', 'quit']
test_res = setOfWordVec(wordVec, test_words)
index = list(range(len(wordVec)))
print(wordVec)
print(index)
print(test_res)

['park', 'posting', 'problems', 'dog', 'food', 'stupid', 'has', 'is', 'mr', 'how', 'help', 'steak', 'not', 'so', 'dalmation', 'quit', 'garbage', 'ate', 'to', 'maybe', 'him', 'flea', 'buying', 'love', 'stop', 'I', 'please', 'licks', 'take', 'worthless', 'cute', 'my']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


## 训练函数

In [11]:
def trainNBS(train_data, train_lables, wordVec):
    ''' set of train labels '''
    label_v = list(set(train_lables))
    label_N = len(label_v)
    vec_len = len(wordVec)
    Ps = np.zeros((label_N, vec_len))
    Ns = np.zeros((label_N, 1))
    for data, label in zip(train_data, train_lables):
        vec = setOfWordVec(wordVec, data)
        idx = label_v.index(label)
        Ps[idx] += np.array(vec)
        Ns[idx] += 1
    return Ps/Ns, Ns.squeeze()/len(train_lables)

In [12]:
train_Pxy, train_Py = trainNBS(dataset, datalabel, wordVec)

In [13]:
print("x|y\n", train_Pxy)
print("y\n", train_Py)

x|y
 [[0.         0.         0.33333333 0.33333333 0.         0.
  0.33333333 0.33333333 0.33333333 0.33333333 0.33333333 0.33333333
  0.         0.33333333 0.33333333 0.         0.         0.33333333
  0.33333333 0.         0.66666667 0.33333333 0.         0.33333333
  0.33333333 0.33333333 0.33333333 0.33333333 0.         0.
  0.33333333 1.        ]
 [0.33333333 0.33333333 0.         0.66666667 0.33333333 1.
  0.         0.         0.         0.         0.         0.
  0.33333333 0.         0.         0.33333333 0.33333333 0.
  0.33333333 0.33333333 0.33333333 0.         0.33333333 0.
  0.33333333 0.         0.         0.         0.33333333 0.66666667
  0.         0.        ]]
y
 [0.5 0.5]


In [24]:
def classifyNBS(testDoc, Pxy, Py, wordVec):
    vec = setOfWordVec(wordVec, testDoc)
    mat = np.dot([vec], Pxy.T) * Py.reshape(1, -1)
    mat = mat.squeeze()
    print(mat)
    idx = np.argmax(mat)
    return idx

In [27]:
''' test naive bayes '''
test_words = ['love', 'my', 'dalmation']
print(classifyNBS(test_words, train_Pxy, train_Py, wordVec))
test_words = ['I', 'stupid']
print(classifyNBS(test_words, train_Pxy, train_Py, wordVec))

[0.83333333 0.        ]
0
[0.16666667 0.5       ]
1
